### Script to Transcribe each audio file 

In [1]:
# importing libraries 
import speech_recognition as sr 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence

# create a speech recognition object
r = sr.Recognizer()

# a function that splits the audio file into chunks
# and applies speech recognition
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_mp3(path)  
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text

In [2]:
path = '../data/audio/clean/' + 'WOL_vs_FUL_2022-08-13.mp3'
path = '../data/audio/clean/' + '2min_test.mp3'

print("\nFull text:", get_large_audio_transcription(path))

FileNotFoundError: [Errno 2] No such file or directory: '../data/audio/clean/2min_test.mp3'

In [ ]:
import whisper 

In [ ]:
model = whisper.load_model("medium.en")
result = model.transcribe('../data/audio/clean/'+str(2)+'min_test.mp3')
print(result["text"])

In [ ]:
from IPython.display import Audio

def play_mp3(filepath):
  # read the MP3 file
  audio = Audio(filepath)

  # display the audio player
  display(audio)
    
play_mp3('../data/audio/clean/'+str(2)+'min_test.mp3')

In [ ]:
import wave
import contextlib
import IPython.display as ipd

filepath = '../data/audio/clean/2min_test.mp3'

# Open the audio file
with contextlib.closing(wave.open(filepath, 'r')) as f:
    # Read the audio file properties
    num_channels = f.getnchannels()
    sample_width = f.getsampwidth()
    frame_rate = f.getframerate()
    num_frames = f.getnframes()

    # Read the audio file data
    audio = f.readframes(num_frames)

# Convert the audio data to a NumPy array
import numpy as np
audio = np.frombuffer(audio, dtype=np.int16)

# Split the audio data into channels
if num_channels == 2:
    audio = audio.reshape((num_frames, 2))
else:
    audio = audio.reshape((num_frames, 1))

# Search the audio data for the name
name = "Premier"
name_audio = None
name_timestamp = None
for i in range(num_frames):
    # Check if the name is present in the audio data at this frame
    if name in str(audio[i]):
        # If the name is found, save the audio data and timestamp
        name_audio = audio[i-50:i+50]
        name_timestamp = i / frame_rate
        break

# If the name was found, play the audio snippet and print the timestamp
if name_audio is not None:
    ipd.display(ipd.Audio(name_audio, rate=frame_rate))
    print(f"Name found at timestamp {name_timestamp:.2f} seconds")
else:
    print("Name not found in audio")


In [ ]:
import speech_recognition as sr
import pronouncing
import difflib
import mutagen.mp3
import contextlib
from utils import *
import whisper
import string

def find_name_in_audio(audio_file, name):
    # Load the MP3 file
#     audio = AudioSegment.from_mp3(audio_file)  

#     print(type(audio))
#     # Transcribe the audio to text
#     r = sr.Recognizer()
    
#     # Create audio data
#     with audio as source:
#         audiodata = recognizer.record(audio)
#     print(type(audiodata))


#     # Create an instance of the Recognizer class
#     r = sr.Recognizer()

#     # Create audio file instance from the original file
#     audio_ex = sr.AudioFile(audio_file)
#     print(type(audio_ex))

#     # Create audio data
#     with audio_ex as source:
#         audiodata = r.record(audio_ex)
#     print(type(audiodata))

    
    
#     text = r.recognize_google(audiodata,language='en-GB',show_all=True)

    model = whisper.load_model("medium.en")
    result = model.transcribe('../data/audio/clean/'+str(2)+'min_test.wav')
    text = result["text"]
    print(text)

    # Transcribe the names to the IPA
    ipa_name = pronouncing.phones_for_word(name)[0]
    ipa_text = []
    for word in text.split(' '):
        ipa_text.append(pronouncing.phones_for_word(word)[0])

    # Find the timestamp of the name in the text
    name_timestamp = None
    for i, word in enumerate(ipa_text):
        # Use fuzzy matching to compare the name and the transcription
        ratio = difflib.SequenceMatcher(None, word, ipa_name).ratio()
        if ratio > 0.8:
            # Calculate the timestamp
            name_timestamp = (i / len(ipa_text)) * audio.duration_seconds

            break

    return name_timestamp


In [ ]:
name = "Premier"
audio_file = '../data/audio/clean/'+'2min_test.wav' #+ '5min_test.wav'

find_name_in_audio(audio_file, name)

In [ ]:
model = whisper.load_model("large-v2")
result = model.transcribe('../data/audio/clean/'+str(2)+'min_test.mp3')
text = result["text"]
print(text)

In [ ]:
model = whisper.load_model("medium.en")
result = model.transcribe('../data/audio/clean/'+str(2)+'min_test.mp3')
text = result["text"]
print(text)

In [ ]:
table = str.maketrans('', '', string.punctuation)
clean_text = [w.strip(string.punctuation).lower() for w in text.split()]
clean_text = [s for s in clean_text for s in s.split('-')]

print(clean_text)

In [ ]:


# Transcribe the names to the IPA
ipa_name = pronouncing.phones_for_word(name)[0]
ipa_text = []
for word in text.split(' '):
    ipa_text.append(pronouncing.phones_for_word(word)[0])

# Find the timestamp of the name in the text
name_timestamp = None
for i, word in enumerate(ipa_text):
    # Use fuzzy matching to compare the name and the transcription
    ratio = difflib.SequenceMatcher(None, word, ipa_name).ratio()
    if ratio > 0.8:
        # Calculate the timestamp
        name_timestamp = (i / len(ipa_text)) * audio.duration_seconds

        break

In [ ]:
print(text.split()[0])

name = 'Premier'

ipa_text = []
for word in clean_text:
    print(word)
    ipa_text.append(pronouncing.phones_for_word(word)[0])
    
pronouncing.phones_for_word(name)[0]

In [ ]:
from googletrans import Translator
translator = Translator()

k = translator.translate("mitrovic ",src='en',dest='en')
k.text

In [9]:
# First
from utils import *

# resave all audio files in clean folder
save_all_mp3_as_wav('../data/audio/clean/',delete=False)


In [10]:
import whisper
import pickle
import os

        
def speech_to_text(src_dir='../data/audio/clean/',dest_dir='../data/text/raw/'):
    import whisper
    model = whisper.load_model("large-v2")
    
    # Iterate through the files in the source directory
    for filename in os.listdir(src_dir):
        # Check if the file is an MP3 file
        if filename.endswith('.wav'):
            # Check if the file exists in the destination directory            
            if not os.path.exists(os.path.join(dest_dir, filename[:-4]+'.txt')):
                
                result = model.transcribe(src_dir+filename,verbose=True)
                text = result["text"]
                
                text_location = dest_dir+filename[:-4]+'.txt'
                print(text_location)
                
                # save text as raw text
                with open(text_location, 'w', encoding='utf-8') as f:
                    print(type(text))
                    f.write(text)

                # save result as pickle
                pickle_location = dest_dir+'pickles/'+filename[:-4]+'.pickle'
                print(pickle_location)
                with open(pickle_location, 'wb') as p:
                    pickle.dump(result, p)
                

                

                
                
                
    
    

In [11]:
speech_to_text()

/opt/homebrew/Caskroom/miniforge/base/envs/CommentaryPrediction/lib/python3.8/site-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:00.000 --> 00:01.000]  in the Premier League.
[00:02.400 --> 00:03.400]  And away we go.
[00:03.400 --> 00:05.600]  Grenfords have done Arsenal a favour.
[00:06.700 --> 00:10.300]  But the value of the B's victory will be really felt
[00:10.400 --> 00:12.800]  if the Gunners can get their own win tonight.
[00:13.300 --> 00:14.100]  The Wolves,
[00:14.300 --> 00:16.800]  they'll hope to get off the bottom after they were put there
[00:17.400 --> 00:20.000]  by Forrest beating Christopher Hannis earlier.
[00:21.600 --> 00:22.700]  Yeah, never
[00:23.500 --> 00:25.900]  a nice day when you drop down without kicking a ball,
[00:25.900 --> 00:26.800]  but they've got a chance
[00:27.300 --> 00:28.500]  tonight, Wolves to
[00:28.500 --> 00:30.300]  change their fortunes.
[00:30.300 --> 00:32.000]  New manager in the stands
[00:32.900 --> 00:33.900]  can often
[00:34.40

[07:01.400 --> 07:03.400]  Arsenal defend a high line.
[07:04.400 --> 07:07.400]  And that should be a source of encouragement
[07:07.400 --> 07:08.400]  for the likes of Traore.
[07:18.400 --> 07:20.400]  He wasn't off by a mile, if you ask.
[07:23.400 --> 07:25.400]  Salibi didn't get the ball.
[07:28.400 --> 07:29.400]  Well, the check is complete.
[07:29.400 --> 07:30.400]  They did have a look.
[07:30.400 --> 07:32.400]  They are Mike Dean.
[07:39.400 --> 07:41.400]  As we know, Arsenal dominating possession.
[07:41.400 --> 07:42.400]  He's opening.
[07:43.400 --> 07:45.400]  Seven minutes or so, but Wolves
[07:45.400 --> 07:46.400]  add an opening.
[07:47.400 --> 07:49.400]  There to get him behind the Arsenal defence.
[07:49.400 --> 07:50.400]  This is Saka.
[07:55.400 --> 07:56.400]  And Saka.
[07:58.400 --> 07:59.400]  Checks back.
[07:59.400 --> 08:00.400]  In from Saka.
[08:00.400 --> 08:01.400]  Not a bad ball.
[08:03.400 --> 08:04.400]  Siencienko.
[08:05.400 --> 08:06.400

[15:24.400 --> 15:31.400]  It's a statement of faith in Fabio Vieira.
[15:31.400 --> 15:37.400]  Mainly, he's run out in Europe.
[15:37.400 --> 15:41.400]  Shown some great technical quality.
[15:41.400 --> 15:45.400]  Playing with Oerdegaard.
[15:49.400 --> 15:51.400]  Back up there.
[15:51.400 --> 15:55.400]  It will be interesting with Fabio Vieira and Oerdegaard in the same side.
[15:55.400 --> 15:59.400]  Two players, you might say, are a similar profile.
[16:03.400 --> 16:07.400]  Thomas Pate in that single pivot at the base of the midfield for Arsenal.
[16:07.400 --> 16:10.400]  Or Zinchenko joining him every now and again.
[16:10.400 --> 16:12.400]  Zinchenko again.
[16:20.400 --> 16:22.400]  Saliba.
[16:32.400 --> 16:34.400]  It's Saliba.
[16:34.400 --> 16:38.400]  Now Gabriel.
[16:38.400 --> 16:40.400]  Saliba again.
[16:43.400 --> 16:45.400]  White.
[16:45.400 --> 16:47.400]  Now Saket.
[16:49.400 --> 16:51.400]  It's Saliba.
[16:51.400 --> 16:53.400]  Zinchenko.
[16:58.400 

[23:29.400 --> 23:31.400]  Fabio down there.
[23:34.400 --> 23:36.400]  He's in it now.
[23:36.400 --> 23:38.400]  Stabel.
[23:41.400 --> 23:43.400]  He knows we're cameras on him at the moment.
[23:46.400 --> 23:48.400]  Sparkling shoes as well.
[23:50.400 --> 23:52.400]  Here's Tillman.
[23:55.400 --> 23:57.400]  Bueno.
[23:57.400 --> 23:59.400]  And Totti was put length by Saka.
[24:05.400 --> 24:07.400]  Well, Seawolves have been pretty happy so far.
[24:08.400 --> 24:12.400]  By the fact that they haven't conceded, and they do carry a threat.
[24:18.400 --> 24:20.400]  That's Bueno.
[24:22.400 --> 24:24.400]  Totti.
[24:24.400 --> 24:26.400]  And then Foutinho.
[24:27.400 --> 24:29.400]  So get that one away to Bueno.
[24:31.400 --> 24:33.400]  They know how they can hurt this Arsenal side, Wolves.
[24:33.400 --> 24:35.400]  They're recognising that.
[24:36.400 --> 24:38.400]  Arteta's recognising it as well.
[24:39.400 --> 24:41.400]  Maybe a little bit of frustration in there.
[

...

[01:08:02.180 --> 01:08:09.180]  The goal is a classic example. The sheer fact that he's in that vicinity and the way he's arriving in the box shows
[01:08:09.180 --> 01:08:14.180]  Mikel Arteta that he can be that player to arrive in the box and maybe even be more of a goal threat.
[01:08:18.180 --> 01:08:23.180]  Now Dakar. Trying to turn inside. Gabriel didn't get much change from him.
[01:08:26.180 --> 01:08:27.180]  Pratt.
[01:08:27.180 --> 01:08:34.180]  And now Maddison. Still desperately need him to conjure something here.
[01:08:39.180 --> 01:08:41.180]  Evans.
[01:08:41.180 --> 01:08:44.180]  And now it's going to be a moment of silence.
[01:08:56.180 --> 01:09:04.180]  Arsenal won't mind this at all. It's going to be these moments in the game where both sides will just want to calm things down in this heat.
[01:09:04.180 --> 01:09:06.180]  Pratt.
[01:09:12.180 --> 01:09:14.180]  Evans.
[01:09:18.180 --> 01:09:20.180]  No changes from Arsenal so far.
[01:09:20.180 --> 01:09:3

[01:15:21.180 --> 01:15:24.180]  The release is Saka.
[01:15:24.180 --> 01:15:26.180]  Djakka is free at the back post.
[01:15:26.180 --> 01:15:28.180]  Here is Bakaya Saka.
[01:15:28.180 --> 01:15:30.180]  And inside to Jesus.
[01:15:30.180 --> 01:15:32.180]  And it's wide.
[01:15:36.180 --> 01:15:38.180]  Nice counterattack from Arsenal.
[01:15:38.180 --> 01:15:39.180]  I like the way Jesus is waiting.
[01:15:39.180 --> 01:15:41.180]  Good head movement.
[01:15:41.180 --> 01:15:42.180]  Looking for his teammates.
[01:15:42.180 --> 01:15:44.180]  Who can he play in?
[01:15:44.180 --> 01:15:46.180]  See Saka wide on the right.
[01:15:46.180 --> 01:15:48.180]  And I thought Saka was going to go alone here,
[01:15:48.180 --> 01:15:49.180]  but he just doesn't.
[01:15:49.180 --> 01:15:51.180]  He's that little pass back into Jesus.
[01:15:51.180 --> 01:15:53.180]  He doesn't strike that clean leap.
[01:15:53.180 --> 01:15:54.180]  See from this angle,
[01:15:54.180 --> 01:15:58.180]  it j

[01:21:35.180 --> 01:21:39.180]  And Arsenal are going to flex their depth as well.
[01:21:40.180 --> 01:21:42.180]  Pierantini, a different kind of ball back.
[01:21:45.180 --> 01:21:47.180]  Interesting reception, wasn't it?
[01:21:47.180 --> 01:21:49.180]  I think the supporters really letting Pierantini know
[01:21:50.180 --> 01:21:52.180]  it's very much a big part of this football club.
[01:21:58.180 --> 01:22:01.180]  And Arsenal now, surely they won't throw it away from here.
[01:22:01.180 --> 01:22:03.180]  He can't speak too quickly in this game.
[01:22:03.180 --> 01:22:04.180]  Here's Ndidi.
[01:22:05.180 --> 01:22:06.180]  France.
[01:22:08.180 --> 01:22:09.180]  And allowed to go.
[01:22:10.180 --> 01:22:11.180]  And here's Maddison.
[01:22:12.180 --> 01:22:15.180]  A bit of second half from him, but he couldn't get past Gabriel.
[01:22:15.180 --> 01:22:17.180]  I think the ball had just gone out of play there.
[01:22:18.180 --> 01:22:19.180]  There will be a corner to Lei

[01:28:15.180 --> 01:28:19.180]  Emil Smith Rowe is back after injury as well.
[01:28:19.180 --> 01:28:23.180]  But most of the support is reserved for Gabriel Jesus,
[01:28:23.180 --> 01:28:26.180]  the Brazilian on debut with two goals.
[01:28:27.180 --> 01:28:31.180]  Chances to score a hat-trick, but he'll have plenty more chances
[01:28:31.180 --> 01:28:33.180]  in an Arsenal shirt, that's for sure.
[01:28:33.180 --> 01:28:36.180]  A very encouraging home debut for the Brazilian.
[01:28:37.180 --> 01:28:38.180]  And Saka off as well.
[01:28:39.180 --> 01:28:41.180]  And Eddie Nketiah.
[01:28:41.180 --> 01:28:45.180]  You can see we'll get plenty of minutes this season because
[01:28:45.180 --> 01:28:47.180]  Gabriel Jesus can't play every game.
[01:28:49.180 --> 01:28:52.180]  And there is the Smith Rowe and Saka chance.
[01:28:53.180 --> 01:28:56.180]  So much for Arsenal fans to be happy about today, but also
[01:28:59.180 --> 01:29:03.180]  moments for Leicester in this game th

[01:34:26.180 --> 01:34:40.180]  Have to go some to find players to improve this side.
[01:34:40.180 --> 01:34:49.180]  The City fans will be hoping the club just makes a outfield signing.
[01:34:49.180 --> 01:34:56.180]  Five minutes to be added on here.
[01:34:56.180 --> 01:34:59.180]  Madison winning a free kick.
[01:34:59.180 --> 01:35:02.180]  The fans desperate for more goals.
[01:35:02.180 --> 01:35:03.180]  Martinelli penalised.
[01:35:03.180 --> 01:35:05.180]  Who is that winning the ball back again as well?
[01:35:05.180 --> 01:35:07.180]  Martinelli.
[01:35:07.180 --> 01:35:12.180]  It's been a massive feature of his game and of Arsenal's game in fact.
[01:35:12.180 --> 01:35:14.180]  We can talk about how well they've done with the ball.
[01:35:14.180 --> 01:35:15.180]  The interchanging.
[01:35:15.180 --> 01:35:19.180]  The little patterns and partnerships all over the pitch and one-twos.
[01:35:19.180 --> 01:35:22.180]  But the midfield and the front three.
[01:35:22.180 

In [ ]:
path = '/Users/keirparker/PycharmProjects/LiveCommentaryPrediction/data/text/raw/pickles/2min_test.pickle'
              
with open(path, 'rb') as f:
    loaded_data = pickle.load(f)

print(loaded_data)